In [153]:
from matplotlib import pyplot as plt
import os
import numpy as np
import json
from sklearn.linear_model import LinearRegression
import pandas as pd


file_dir = "/data03/sunyi/time_constrained_cot/outputs/2_6"
latency_path = "/home/sunyi/CoT/Time-Constrained-CoT/latency/0210"
model_list = [
    "Qwen/QwQ-32B-Preview",
    # "Skywork/Skywork-o1-Open-Llama-3.1-8B", 
    # "PowerInfer/SmallThinker-3B-Preview",
    "NovaSky-AI/Sky-T1-32B-Preview",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "Qwen/Qwen2.5-Math-1.5B-Instruct",
    "Qwen/Qwen2.5-Math-7B-Instruct",
    "mistralai/Mathstral-7B-v0.1",
    "Qwen/Qwen2.5-32B-Instruct",
    "Qwen/Qwen2.5-14B-Instruct",
    "Qwen/Qwen2.5-7B-Instruct",
    "Qwen/Qwen2.5-3B-Instruct",
    "Qwen/Qwen2.5-1.5B-Instruct",
    "mistralai/Mistral-Small-Instruct-2409",
    "mistralai/Mistral-Nemo-Instruct-2407",
    "mistralai/Ministral-8B-Instruct-2410",
    "google/gemma-2-27b-it",
    "google/gemma-2-9b-it",
    "google/gemma-2-2b-it",
    "microsoft/Phi-3-medium-128k-instruct",
    "microsoft/Phi-3-small-128k-instruct",
    "microsoft/Phi-3-mini-128k-instruct",
    "microsoft/Phi-3.5-mini-instruct",
    "microsoft/phi-4",
    "meta-llama/Llama-3.1-8B-Instruct",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.2-1B-Instruct",
]

In [154]:
PROMP_LIST = [ "-aav", "-c2f", "-sbs"]


MODEL_SERIES_MAP = {
    "Qwen/QwQ-32B-Preview": "qwen",
    "Qwen/Qwen2.5-32B-Instruct": "qwen",
    "Qwen/Qwen2.5-14B-Instruct": "qwen",
    "Qwen/Qwen2.5-7B-Instruct": "qwen",
    "Qwen/Qwen2.5-3B-Instruct": "qwen",
    "Qwen/Qwen2.5-1.5B-Instruct": "qwen",
    "Qwen/Qwen2.5-Math-1.5B-Instruct": "qwen-math",
    "Qwen/Qwen2.5-Math-7B-Instruct": "qwen-math",
    "internlm/internlm2_5-1_8b-chat": "internlm",
    "internlm/internlm2_5-7b-chat": "internlm",
    "internlm/internlm2_5-20b-chat": "internlm",
    "google/gemma-2-2b-it": "gemma",
    "google/gemma-2-9b-it": "gemma",
    "google/gemma-2-27b-it": "gemma",
    "mistralai/Mathstral-7B-v0.1": "mistral",
    "mistralai/Ministral-8B-Instruct-2410": "mistral",
    "mistralai/Mistral-Nemo-Instruct-2407": "mistral",
    "mistralai/Mistral-Small-Instruct-2409": "mistral",
    "microsoft/phi-4": "phi4",
    "microsoft/Phi-3-medium-128k-instruct": "phi3medium",
    "microsoft/Phi-3-small-128k-instruct": "phi3small",
    "microsoft/Phi-3.5-mini-instruct": "phi3mini",
    "microsoft/Phi-3-mini-128k-instruct": "phi3mini",
    "NovaSky-AI/Sky-T1-32B-Preview": "qwen",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B": "deepseek-r1-distill",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B": "deepseek-r1-distill",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B": "deepseek-r1-distill",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B": "deepseek-r1-distill",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B": "deepseek-r1-distill",
    "meta-llama/Llama-3.2-3B-Instruct": "llama",
    "meta-llama/Llama-3.2-1B-Instruct": "llama",
    "meta-llama/Llama-3.1-8B-Instruct": "llama"
}


MODEL_SERIES_PROMPT_TYPE_MAP = {
    "qwen": ["qwen" + prompt for prompt in PROMP_LIST],
    "qwen-math": ["qwen-math" + prompt for prompt in PROMP_LIST],
    "internlm": ["internlm"+prompt for prompt in PROMP_LIST],
    "mistral": ["mistral"+prompt for prompt in PROMP_LIST],
    "gemma": ["gemma"+prompt for prompt in PROMP_LIST],
    "phi3mini": ["phi3mini"+prompt for prompt in PROMP_LIST],
    "phi3small": ["phi3small"+prompt for prompt in PROMP_LIST],
    "phi3medium": ["phi3medium"+prompt for prompt in PROMP_LIST],
    "phi4": ["phi4"+prompt for prompt in PROMP_LIST],
    "deepseek-r1-distill": ["deepseek-r1-distill"+prompt for prompt in PROMP_LIST],
    "llama": ["llama"+prompt for prompt in PROMP_LIST]
}

In [155]:
dataset = "gsm8k"
something = "_-1_seed0_t0.0_s0_e-1"

In [156]:
import pandas as pd
import numpy as np

# Select_latency = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]  
Select_latency = list(np.arange(5, 40.5, 0.5))

df = pd.read_csv('model_performance_data.csv', names=['Model', 'Prompt_Type', 'Budget_Index', 'Latency', 'Accuracy'])
# print(df)
model_spe_acc_dicts = {}
for model in model_list:
    model_acc_dict = {}
    for prompt_type in MODEL_SERIES_PROMPT_TYPE_MAP[MODEL_SERIES_MAP[model]]:
        print(model, prompt_type)   
        acc_list = df[(df['Model'] == model.split('/')[-1]) & (df['Prompt_Type'] == prompt_type)]["Accuracy"].tolist()
        latency_list = df[(df['Model'] == model.split('/')[-1]) & (df['Prompt_Type'] == prompt_type)]["Latency"].tolist()
        bugdet_index_list = df[(df['Model'] == model.split('/')[-1]) & (df['Prompt_Type'] == prompt_type)]["Budget_Index"].tolist()
        model_lat_dict = {}
        for s_latency in Select_latency:
            for latency in latency_list:
                current_budget_index = bugdet_index_list[latency_list.index(latency)]
                current_budget_index = int(current_budget_index)
                if float(latency) <= s_latency and current_budget_index != int(bugdet_index_list[-1]) and float(latency_list[current_budget_index+1]) > s_latency:
                    model_lat_dict[s_latency] = float(acc_list[current_budget_index]) + (float(acc_list[current_budget_index+1]) - float(acc_list[current_budget_index])) * (s_latency - float(latency_list[current_budget_index])) / (float(latency_list[current_budget_index+1]) - float(latency_list[current_budget_index])) 
                    break
                else:
                    model_lat_dict[s_latency] = acc_list[current_budget_index]
        model_acc_dict[prompt_type] = model_lat_dict
    model_spe_acc_dicts[model] = model_acc_dict

print(model_spe_acc_dicts)  

data = []

for model in model_spe_acc_dicts.keys():
    acc_dict = model_spe_acc_dicts[model]
    for prompt_type in model_spe_acc_dicts[model].keys():
        acc_list = acc_dict[prompt_type]
        for latency in acc_list.keys():
            data.append({
                'Model': model,
                'Prompt_Type': prompt_type,
                'Latency': latency,
                'Accuracy': acc_list[latency]
            })

# Convert to DataFrame and save
df = pd.DataFrame(data)
df.to_csv('paticular_latency.csv', index=False)

Qwen/QwQ-32B-Preview qwen-aav
Qwen/QwQ-32B-Preview qwen-c2f
Qwen/QwQ-32B-Preview qwen-sbs
NovaSky-AI/Sky-T1-32B-Preview qwen-aav
NovaSky-AI/Sky-T1-32B-Preview qwen-c2f
NovaSky-AI/Sky-T1-32B-Preview qwen-sbs
deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B deepseek-r1-distill-aav
deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B deepseek-r1-distill-c2f
deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B deepseek-r1-distill-sbs
deepseek-ai/DeepSeek-R1-Distill-Qwen-7B deepseek-r1-distill-aav
deepseek-ai/DeepSeek-R1-Distill-Qwen-7B deepseek-r1-distill-c2f
deepseek-ai/DeepSeek-R1-Distill-Qwen-7B deepseek-r1-distill-sbs
deepseek-ai/DeepSeek-R1-Distill-Qwen-14B deepseek-r1-distill-aav
deepseek-ai/DeepSeek-R1-Distill-Qwen-14B deepseek-r1-distill-c2f
deepseek-ai/DeepSeek-R1-Distill-Qwen-14B deepseek-r1-distill-sbs
deepseek-ai/DeepSeek-R1-Distill-Qwen-32B deepseek-r1-distill-aav
deepseek-ai/DeepSeek-R1-Distill-Qwen-32B deepseek-r1-distill-c2f
deepseek-ai/DeepSeek-R1-Distill-Qwen-32B deepseek-r1-distill-sbs
deepseek-ai/D